## Generate SGD-TGT

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import json

from utils_attack import uap_batch
from utils_model import *

dir_results = './results/'
dir_uap = './uaps/'

In [2]:
nb_epoch = 10
atk_name = 'sgd-tgt'

In [3]:
model_names = ['resnet18',
               'resnet18_sfP',
               'resnet18_sfP-mixup',
               'resnet18_sfP-cutout',
               'resnet18_P2-0.3',
               'resnet18_P3-0.3',
               'resnet18_P4-0.3']
#                'resnet20_Q2',
#                'resnet20_Q3',
#                'resnet20_Q4']

In [4]:
def eval_uap(uap):    
    top, top_probs, top1acc, top5acc, outputs, labels = evaluate(model, testloader, uap = uap)

    print('Top 1 accuracy', sum(top1acc) / len(labels))
    print('Top 5 accuracy', sum(top5acc) / len(labels))

    dict_entry = {}
    dict_entry['model_name'] = model_name
    dict_entry['dataset'] = dataset

    dict_entry['atk_name'] = atk_name
    dict_entry['atk_param'] = y_target
    dict_entry['eps'] = eps
    if atk_name[:3] == 'sgd': dict_entry['nb_epoch'] = nb_epoch

    dict_entry['top1acc'] = sum(top1acc) / len(labels)
    dict_entry['top5acc'] = sum(top5acc) / len(labels)
    dict_entry['UER'] = 1 - sum(top1acc) / len(labels)
    print('UER ', dict_entry['UER'])
    print('\n')
    if y_target >= 0: dict_entry['tgt_success'] = sum(outputs == y_target) / len(labels)

    # Output distribution
    for i in range(10):
        dict_entry['label_dist%i' % i] = sum(outputs == i) / len(labels)

    all_results.append(dict_entry)

### Generate UAP

### Evaluate UAP

### Evaluate UAP (Quantized)

In [5]:
Q_param = 4
model_name = 'resnet20_Q%i' % Q_param

In [6]:
all_results = []
batch_size = 500

for dataset in ['cifar10', 'svhn']:
    testloader = get_testdata(dataset, batch_size)
    model = get_model(model_name, dataset, batch_size)

    for y_target in range(10):
        for eps in range(8, 13, 2):

            # Save directory
            uap_save_path = dir_uap + dataset + '/' + model_name
            if not os.path.exists(uap_save_path):
                os.mkdir(uap_save_path)

            # Compute UAP
            save_name = uap_save_path + '/sgd-tgt%i-eps%i.pth' % (y_target, eps)
            print(save_name)
            if os.path.isfile(save_name):
                print('UAP already generated')
            else:
                if dataset == 'svhn': uap, losses = uap_batch(model, testloader, nb_epoch, eps / 255, beta = 8, y_target = y_target, cut_end = True)
                else: uap, losses = uap_batch(model, testloader, nb_epoch, eps / 255, beta = 8, y_target = y_target)
                torch.save(uap, save_name)

Files already downloaded and verified
Loading cifar10/resnet20_Q4.pth
./uaps/cifar10/resnet20_Q4/sgd-tgt0-eps8.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt0-eps10.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt0-eps12.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt1-eps8.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt1-eps10.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt1-eps12.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt2-eps8.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt2-eps10.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt2-eps12.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt3-eps8.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt3-eps10.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt3-eps12.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt4-eps8.pth
UAP already generated
./uaps/cifar10/resnet20_Q4/sgd-tgt4-eps10.pth
UAP a

In [7]:
batch_size = None

for dataset in ['cifar10', 'svhn']:
    all_results = []
    
    testloader = get_testdata(dataset)
    model = get_model(model_name, dataset)
    model.eval()

    for y_target in range(10):
        for eps in range(8, 13, 2):
            load_path = dir_uap + dataset + '/' + model_name + '/sgd-tgt%i-eps%i.pth' % (y_target, eps)
            print(load_path)
            uap = torch.load(load_path)
            eval_uap(uap)

        pd.DataFrame(all_results).to_csv(dir_results + 'evalQ%i_tgt-whitebox-%s.csv' % (Q_param, dataset))

Files already downloaded and verified
Loading cifar10/resnet20_Q4.pth
./uaps/cifar10/resnet20_Q4/sgd-tgt0-eps8.pth
 Completed: 0.00%
Top 1 accuracy 0.6237
Top 5 accuracy 0.9338
UER  0.37629999999999997


./uaps/cifar10/resnet20_Q4/sgd-tgt0-eps10.pth
 Completed: 0.00%
Top 1 accuracy 0.5807
Top 5 accuracy 0.91
UER  0.4193


./uaps/cifar10/resnet20_Q4/sgd-tgt0-eps12.pth
 Completed: 0.00%
Top 1 accuracy 0.4905
Top 5 accuracy 0.8791
UER  0.5095000000000001


./uaps/cifar10/resnet20_Q4/sgd-tgt1-eps8.pth
 Completed: 0.00%
Top 1 accuracy 0.5924
Top 5 accuracy 0.9248
UER  0.40759999999999996


./uaps/cifar10/resnet20_Q4/sgd-tgt1-eps10.pth
 Completed: 0.00%
Top 1 accuracy 0.4914
Top 5 accuracy 0.8747
UER  0.5085999999999999


./uaps/cifar10/resnet20_Q4/sgd-tgt1-eps12.pth
 Completed: 0.00%
Top 1 accuracy 0.3913
Top 5 accuracy 0.8331
UER  0.6087


./uaps/cifar10/resnet20_Q4/sgd-tgt2-eps8.pth
 Completed: 0.00%
Top 1 accuracy 0.6337
Top 5 accuracy 0.9491
UER  0.36629999999999996


./uaps/cifar10/res

 Completed: 0.00%
Top 1 accuracy 0.5693377381684082
Top 5 accuracy 0.892440073755378
UER  0.43066226183159184


./uaps/svhn/resnet20_Q4/sgd-tgt9-eps10.pth
 Completed: 0.00%
Top 1 accuracy 0.47503073140749846
Top 5 accuracy 0.8254840196681008
UER  0.5249692685925016


./uaps/svhn/resnet20_Q4/sgd-tgt9-eps12.pth
 Completed: 0.00%
Top 1 accuracy 0.36639520590043023
Top 5 accuracy 0.7742393976644131
UER  0.6336047940995697


